<a href="https://colab.research.google.com/github/yoramabbou770/AICourseRoad2/blob/main/Task7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unzip my zip folder, the path is : /content/drive/MyDrive/cats_and_dogs_filtered.zip

In [1]:
import zipfile
import os

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/cats_and_dogs_filtered.zip'

# Directory to extract the contents
extracted_dir = '/content/cats_and_dogs_filtered'

# Check if the directory already exists, if not, create it
if not os.path.exists(extracted_dir):
    os.makedirs(extracted_dir)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Display the list of files and directories in the extracted folder
print("Contents of the extracted folder:")
print(os.listdir(extracted_dir))

Contents of the extracted folder:
['cats_and_dogs_filtered']


Give me in python and Keras for google colab a model that classify if its a dog or a cat, use the VGG16 model, freeze the top layers, just let fit from the last convolution layer to the end classification layer included.
the path are : /content/cats_and_dogs_filtered/cats_and_dogs_filtered/train and /content/cats_and_dogs_filtered/cats_and_dogs_filtered/validation

In [8]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import models, layers, optimizers

# Set the path to your extracted dataset
train_dataset_path = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/train'
validation_dataset_path = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

# Define parameters
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 32

# Create VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model for classification
model = models.Sequential()

# Add the VGG16 convolutional base
model.add(base_model)

# Add Flatten layer to match the PyTorch model's input shape
model.add(layers.Flatten(input_shape=(7, 7, 512)))

# Add fully connected layers matching the PyTorch model
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))  # 1 neuron for binary classification (cat or dog)

# Compile the model
model.compile(optimizer=optimizers.RMSprop(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation and preparation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # binary classification
    classes=['cats', 'dogs'],  # class labels
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['cats', 'dogs'],
)

# Calculate steps_per_epoch and validation_steps
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size

# Train the model with validation data
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/5
62/62 [==============================] - 35s 545ms/step - loss: 0.6286 - accuracy: 0.6712 - val_loss: 0.3480 - val_accuracy: 0.8478
Epoch 2/5
62/62 [==============================] - 32s 510ms/step - loss: 0.4055 - accuracy: 0.8095 - val_loss: 0.3049 - val_accuracy: 0.8669
Epoch 3/5
62/62 [==============================] - 34s 545ms/step - loss: 0.3341 - accuracy: 0.8476 - val_loss: 0.2363 - val_accuracy: 0.9032
Epoch 4/5
62/62 [==============================] - 36s 584ms/step - loss: 0.3073 - accuracy: 0.8674 - val_loss: 0.2260 - val_accuracy: 0.9083
Epoch 5/5
62/62 [==============================] - 32s 519ms/step - loss: 0.2962 - accuracy: 0.8613 - val_loss: 0.2103 - val_accuracy: 0.9113


Without freeze the top layers

In [10]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import models, layers, optimizers

# Set the path to your extracted dataset
train_dataset_path = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/train'
validation_dataset_path = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

# Define parameters
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 32

# Create VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Create a new model for classification
model = models.Sequential()

# Add the VGG16 convolutional base
model.add(base_model)

# Add Flatten layer to match the PyTorch model's input shape
model.add(layers.Flatten(input_shape=(7, 7, 512)))

# Add fully connected layers matching the PyTorch model
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))  # 1 neuron for binary classification (cat or dog)

# Compile the model
model.compile(optimizer=optimizers.RMSprop(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation and preparation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # binary classification
    classes=['cats', 'dogs'],  # class labels
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['cats', 'dogs'],
)

# Calculate steps_per_epoch and validation_steps
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size

# Train the model with validation data
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/5
62/62 [==============================] - 67s 820ms/step - loss: 0.5960 - accuracy: 0.6712 - val_loss: 0.2774 - val_accuracy: 0.8982
Epoch 2/5
62/62 [==============================] - 42s 659ms/step - loss: 0.2546 - accuracy: 0.8943 - val_loss: 0.1074 - val_accuracy: 0.9607
Epoch 3/5
62/62 [==============================] - 43s 695ms/step - loss: 0.1369 - accuracy: 0.9522 - val_loss: 0.0827 - val_accuracy: 0.9657
Epoch 4/5
62/62 [==============================] - 42s 679ms/step - loss: 0.1135 - accuracy: 0.9517 - val_loss: 0.0710 - val_accuracy: 0.9798
Epoch 5/5
62/62 [==============================] - 41s 654ms/step - loss: 0.0769 - accuracy: 0.9670 - val_loss: 0.0691 - val_accuracy: 0.9718


Give me the equivalent code for the PyTorch model

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from tqdm import tqdm

# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data = ImageFolder(data_path, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Define the PyTorch model with frozen top layers
class CustomVGG16(nn.Module):
    def __init__(self):
        super(CustomVGG16, self).__init__()
        vgg16 = models.vgg16(pretrained=True)

        # Freeze the top layers
        for param in vgg16.features.parameters():
            param.requires_grad = False

        # Extract the layers from the last convolutional layer to the end of the classifier
        classifier_layers = list(vgg16.classifier.children())[-6:]

        self.features = vgg16.features
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # Adjusted input size to match flattened output
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Set the path to your extracted dataset
train_dataset_path = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/train'
validation_dataset_path = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

# Define parameters
img_width, img_height = 224, 224
input_shape = (3, img_width, img_height)  # PyTorch uses (channels, height, width)
batch_size = 32

# Define data transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((img_width, img_height)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

validation_transform = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create custom datasets and data loaders
train_dataset = CustomDataset(train_dataset_path, transform=train_transform)
validation_dataset = CustomDataset(validation_dataset_path, transform=validation_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Create the PyTorch model
pytorch_model = CustomVGG16()

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.RMSprop(pytorch_model.parameters(), lr=2e-5)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    pytorch_model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        optimizer.zero_grad()
        outputs = pytorch_model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    average_loss = running_loss / len(train_loader)
    print(f'Training Loss: {average_loss}')

# Validation loop
pytorch_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in tqdm(validation_loader, desc='Validation'):
        outputs = pytorch_model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels.float().view(-1, 1)).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy}')

Epoch 1/5: 100%|██████████| 63/63 [21:08<00:00, 20.13s/it]


Training Loss: 0.23582229550395692


Epoch 2/5: 100%|██████████| 63/63 [20:54<00:00, 19.92s/it]


Training Loss: 0.115681962716201


Epoch 3/5: 100%|██████████| 63/63 [20:36<00:00, 19.63s/it]


...

Training Loss: 0.11370683402296096


Epoch 5/5: 100%|██████████| 63/63 [21:10<00:00, 20.17s/it]


Training Loss: 0.10982418229566916


Validation: 100%|██████████| 32/32 [09:47<00:00, 18.35s/it]

Validation Accuracy: 0.988
